# Data Collection

### Import required libraries

In [24]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os
import pandas as pd
from uuid import uuid1
from datetime import datetime
from datetime import date

### Display live camera feed

In [25]:
image_resolution = 224

camera = Camera.instance(width=image_resolution, height=image_resolution, fps = 2)

image = widgets.Image(format='jpeg', width=image_resolution, height=image_resolution)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### Create directories + dataframe to store the data

In [26]:
image_param_column_list = ['UUID','DATE','TIME','RUNWAY','ROOM', 'SIDE', 'MARK', 'FLOOR', ' LIGHT', 'BACK', 'LONG DIST', 'LAT DIST', 'ANGLE']
image_param_df = pd.DataFrame(columns=image_param_column_list)

runway_dir = 'dataset/runway'
norunway_dir = 'dataset/norunway'
partrunway_dir = 'dataset/partrunway'

try:
    os.makedirs(norunway_dir)
    os.makedirs(partrunway_dir)
    os.makedirs(runway_dir)
except FileExistsError:
    print('Directories not created because they already exist')

### Initialise variables

In [27]:
longitudinal_distance_range = 120
lateral_distance_range = 120
axis_angle_range = 90
longitudinal_distance = 0
lateral_distance = 0
axis_angle = 0

button_layout = widgets.Layout(width='128px', height='64px')
text_layout = widgets.Layout(width='200px', height='64px')

### Prepare Interface

In [28]:
# General parameters setting interface
marking_selection = widgets.RadioButtons(
    options=[('Light',0), ('Dark',1)],
    description='Marking:',
    disabled=False
)
floor_selection = widgets.RadioButtons(
    options=[('Light',0), ('Dark',1)],
    description='Floor:',
    disabled=False
)
lighting_selection = widgets.RadioButtons(
    options=[('Light',0), ('Dark',1)],
    description='Lighting:',
    disabled=False
)
background_selection = widgets.RadioButtons(
    options=[('Minimalist',0), ('Clutered',1)],
    description='Background:',
    disabled=False
)
side_selection = widgets.RadioButtons(
    options=[('LHS',0), ('RHS',1)],
    description='Runway side:',
    disabled=False
)
room_selection = widgets.Dropdown(
    options=[('Living',0), ('Dining',1),('Kitchen',2), 
            ('Mudroom',3), ('Corridor',4),('Bathroom',5),
            ('StudyC',6), ('StudyA',7),('Bedroom',8),
            ('Backroom',9), ('Balcony',10)],
    value=7,
    description='Room:',
    disabled=False,
)

# Grid parameters setting interface
longitudinal_distance_display = widgets.IntText(description='Longitudinal distance', layout=text_layout, value=longitudinal_distance)
lateral_distance_display = widgets.IntText(description='Lateral distance', layout=text_layout, value=lateral_distance)
axis_angle_display = widgets.IntText(description='Axis angle', layout=text_layout, value=axis_angle)

# Data collection buttons interface
norunway_button = widgets.Button(description='No runway', button_style='danger', layout=button_layout)
partrunway_button = widgets.Button(description='Partial runway', button_style='warning', layout=button_layout)
runway_button = widgets.Button(description='Full runway', button_style='success', layout=button_layout)
pass_button = widgets.Button(description='Pass',button_style='info', layout=button_layout)

norunway_count = widgets.IntText(layout=button_layout, value=len(os.listdir(norunway_dir)))
partrunway_count = widgets.IntText(layout=button_layout, value=len(os.listdir(partrunway_dir)))
runway_count = widgets.IntText(layout=button_layout, value=len(os.listdir(runway_dir)))


### Define functions

In [29]:
def save_snapshot(directory, fileid):
    image_path = os.path.join(directory, str(fileid) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_parameters(fid, date, time, runway, room, side, marking, floor, lighting, background, long_dist, lat_dist, angle):
    global image_param_column_list, image_param_df
    temp_df = pd.DataFrame([[fid, date, time, runway, room, side, marking, floor, lighting, background, long_dist, lat_dist, angle]], columns=image_param_column_list)
    image_param_df = image_param_df.append(temp_df)
    pass

def grid_pattern():
    global longitudinal_distance, lateral_distance, axis_angle
    cycle_completed = 0
    if longitudinal_distance == longitudinal_distance_range:
        longitudinal_distance = 0
        if lateral_distance == lateral_distance_range:
            lateral_distance = 0
            if axis_angle == axis_angle_range:
                axis_angle = 0
            else:
                axis_angle = axis_angle + 45
        else:
            lateral_distance = lateral_distance + 30
    else:
        longitudinal_distance = longitudinal_distance + 30
    longitudinal_distance_display.value = longitudinal_distance
    lateral_distance_display.value = lateral_distance
    axis_angle_display.value = axis_angle

def save_norunway():
    global norunway_dir, norunway_count
    global marking_selection, floor_selection, lighting_selection, background_selection
    global longitudinal_distance, lateral_distance, axis_angle
    fileid = uuid1()
    date_now = str(date.today())
    time_now = datetime.now().strftime("%H:%M:%S")
    save_snapshot(norunway_dir, fileid)
    save_parameters(fileid, date_now, time_now, 0, room_selection.value, side_selection.value, marking_selection.value, floor_selection.value, lighting_selection.value, background_selection.value, longitudinal_distance, lateral_distance, axis_angle)
    norunway_count.value = len(os.listdir(norunway_dir))
    grid_pattern()

def save_partrunway():
    global partrunway_dir, partrunway_count
    global marking_selection, floor_selection, lighting_selection, background_selection
    global longitudinal_distance, lateral_distance, axis_angle
    fileid = uuid1()
    date_now = str(date.today())
    time_now = datetime.now().strftime("%H:%M:%S")
    save_snapshot(partrunway_dir, fileid)
    save_parameters(fileid, date_now, time_now, 1, room_selection.value, side_selection.value, marking_selection.value, floor_selection.value, lighting_selection.value, background_selection.value, longitudinal_distance, lateral_distance, axis_angle)
    partrunway_count.value = len(os.listdir(partrunway_dir))
    grid_pattern()

def save_runway():
    global runway_dir, runway_count
    global marking_selection, floor_selection, lighting_selection, background_selection
    global longitudinal_distance, lateral_distance, axis_angle
    fileid = uuid1()
    date_now = str(date.today())
    time_now = datetime.now().strftime("%H:%M:%S")
    save_snapshot(runway_dir, fileid)
    save_parameters(fileid, date_now, time_now, 2, room_selection.value, side_selection.value, marking_selection.value, floor_selection.value, lighting_selection.value, background_selection.value, longitudinal_distance, lateral_distance, axis_angle)
    runway_count.value = len(os.listdir(runway_dir))
    grid_pattern()
    

def save_pass():
    global runway_marking_selection, floor_selection, lighting_selection, background_selection
    global longitudinal_distance, lateral_distance, axis_angle
    date_now = str(date.today())
    time_now = datetime.now().strftime("%H:%M:%S")    
    save_parameters('na', date_now, time_now, 0, room_selection.value, side_selection.value, marking_selection.value, floor_selection.value, lighting_selection.value, background_selection.value, longitudinal_distance, lateral_distance, axis_angle)
    grid_pattern()

### Collect data

In [31]:
display(widgets.HBox([marking_selection, floor_selection]))
display(widgets.HBox([lighting_selection, background_selection]))
display(widgets.HBox([room_selection, side_selection]))

display(widgets.HBox([longitudinal_distance_display, lateral_distance_display, axis_angle_display]))

display(widgets.HBox([runway_button, runway_count, pass_button]))
display(widgets.HBox([partrunway_button, partrunway_count]))
display(widgets.HBox([norunway_button,norunway_count]))

display(image, width = image_resolution, height = image_resolution)

norunway_button.on_click(lambda x: save_norunway())
partrunway_button.on_click(lambda x: save_partrunway())
runway_button.on_click(lambda x: save_runway())
pass_button.on_click(lambda x: save_pass())


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### Save Image parameters to dedicated .csv file

In [32]:
now = datetime.now().strftime("%d%m%Y_%H%M%S")
image_param_df.to_csv('dataset/Image_Parameters_' + now + '.csv')


### Zip dataset folder for export

In [34]:
!zip -r -q dataset.zip dataset